In [ ]:
# 🎬 SkyReels V2 - Infinite-Length Film Generator

Welcome to SkyReels V2! This notebook provides an easy-to-use interface for generating high-quality videos using the SkyReels V2 models.

**Features:**
- Text-to-Video Generation
- Image-to-Video Generation  
- Diffusion Forcing for Long Videos
- Multiple Resolution Support (540P/720P)
- Interactive Web Interface

---


In [ ]:
## 🚀 Setup and Installation

First, let's install all required dependencies and clone the repository.


In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq ffmpeg libsm6 libxext6 libxrender-dev libglib2.0-0

# Clone repository
!git clone https://github.com/SkyworkAI/SkyReels-V2.git
%cd SkyReels-V2

# Install Python dependencies
!pip install -q torch==2.5.1 torchvision==0.20.1
!pip install -q diffusers>=0.31.0 transformers==4.49.0 tokenizers==0.21.1
!pip install -q accelerate==1.6.0 tqdm imageio easydict ftfy
!pip install -q opencv-python==4.10.0.84 imageio-ffmpeg
!pip install -q numpy>=1.23.5,<2
!pip install -q gradio spaces
!pip install -q huggingface_hub

# Install video processing dependencies
!pip install -q decord
!pip install -q av

# Install xfuser for multi-GPU support (optional)
!pip install -q xfuser

print("✅ Installation completed!")


In [ ]:
## 🔧 Quick Fix for Missing Dependencies

If you encounter "ModuleNotFoundError" for decord or other modules, run this cell:


In [ ]:
# Quick fix for missing dependencies
print("🔧 Installing missing video processing dependencies...")

# Install decord and other video dependencies
!pip install -q decord av
!pip install -q dashscope  # Also needed for some pipelines

# Verify installations
try:
    import decord
    print("✅ decord installed successfully")
except ImportError:
    print("❌ decord installation failed")

try:
    import av
    print("✅ av installed successfully") 
except ImportError:
    print("❌ av installation failed")

print("🔄 Please restart runtime if issues persist: Runtime → Restart runtime")


In [ ]:
## 💽 Disk Space Management

Google Colab has limited disk space (~112GB). Large AI models can quickly fill this up. Use these tools to manage space:


In [ ]:
import os
import shutil
import subprocess

def check_disk_usage():
    """Check current disk usage and identify large files/folders"""
    print("=== Disk Usage Check ===")
    
    # Get overall disk usage
    total, used, free = shutil.disk_usage('/')
    print(f"💽 Total: {total//1024**3:.1f}GB")
    print(f"📊 Used: {used//1024**3:.1f}GB ({used/total*100:.1f}%)")
    print(f"🆓 Free: {free//1024**3:.1f}GB")
    
    if free < 5*1024**3:  # Less than 5GB free
        print("⚠️  WARNING: Low disk space!")
    
    print("\n=== Largest Folders ===")
    # Find largest folders
    try:
        result = subprocess.run(['du', '-sh', '/content/*'], 
                              capture_output=True, text=True, shell=True)
        if result.stdout:
            lines = result.stdout.strip().split('\n')
            # Sort by size (rough sorting by first character/number)
            for line in sorted(lines, reverse=True)[:10]:
                if line.strip():
                    print(f"📁 {line}")
    except:
        print("Could not get folder sizes")

def cleanup_colab_space():
    """Clean up common space-wasting files in Colab"""
    print("=== Cleaning Up Disk Space ===")
    
    cleanup_commands = [
        # Clear pip cache
        "pip cache purge",
        # Clear apt cache  
        "apt-get clean",
        # Remove temp files
        "rm -rf /tmp/*",
        # Clear Python cache
        "find /content -name '__pycache__' -type d -exec rm -rf {} +",
        # Clear .pyc files
        "find /content -name '*.pyc' -delete",
    ]
    
    for cmd in cleanup_commands:
        try:
            print(f"🧹 Running: {cmd}")
            os.system(cmd)
        except:
            continue
    
    print("✅ Basic cleanup completed!")

def free_model_cache():
    """Clear model caches and temporary files"""
    print("=== Clearing Model Caches ===")
    
    # Clear HuggingFace cache
    hf_cache_dir = "/root/.cache/huggingface"
    if os.path.exists(hf_cache_dir):
        try:
            shutil.rmtree(hf_cache_dir)
            print(f"🗑️  Cleared HuggingFace cache: {hf_cache_dir}")
        except:
            print("❌ Could not clear HuggingFace cache")
    
    # Clear torch cache
    torch_cache_dir = "/root/.cache/torch"
    if os.path.exists(torch_cache_dir):
        try:
            shutil.rmtree(torch_cache_dir)
            print(f"🗑️  Cleared PyTorch cache: {torch_cache_dir}")
        except:
            print("❌ Could not clear PyTorch cache")
    
    # Clear outputs folder if exists
    if os.path.exists("outputs"):
        size = sum(os.path.getsize(os.path.join("outputs", f)) 
                  for f in os.listdir("outputs"))
        print(f"📁 outputs folder: {size//1024**2:.1f}MB")
        
        # Ask before deleting outputs
        response = input("Delete generated videos in outputs/? (y/n): ")
        if response.lower() == 'y':
            shutil.rmtree("outputs")
            os.makedirs("outputs", exist_ok=True)
            print("🗑️  Cleared outputs folder")

# Run disk check
check_disk_usage()


In [ ]:
### 🆘 Emergency Disk Cleanup

If your disk is full, run these commands immediately:


In [ ]:
# 🆘 EMERGENCY CLEANUP - Run this if disk is full!

print("🚨 Emergency disk cleanup starting...")

# 1. Clear all caches immediately
!pip cache purge
!apt-get clean

# 2. Remove temporary files
!rm -rf /tmp/*
!rm -rf /var/tmp/*

# 3. Clear Python caches
!find /content -name '__pycache__' -type d -exec rm -rf {} + 2>/dev/null
!find /content -name '*.pyc' -delete 2>/dev/null

# 4. Clear HuggingFace cache (models will re-download when needed)
!rm -rf /root/.cache/huggingface/

# 5. Clear PyTorch cache
!rm -rf /root/.cache/torch/

# 6. Remove any existing outputs
!rm -rf outputs/

# 7. Clear notebook checkpoints
!find /content -name '.ipynb_checkpoints' -type d -exec rm -rf {} + 2>/dev/null

print("✅ Emergency cleanup completed!")

# Check space after cleanup
total, used, free = shutil.disk_usage('/')
print(f"🆓 Free space after cleanup: {free//1024**3:.1f}GB")

if free < 2*1024**3:  # Still less than 2GB
    print("⚠️  Still low on space! Consider:")
    print("   - Restarting runtime completely")
    print("   - Using smaller models (1.3B instead of 14B)")
    print("   - Generating shorter videos")
    print("   - Upgrading to Colab Pro for more space")


In [ ]:
## 🔧 Quick Fix for Module Import Issues

If you get "No module named 'skyreels_v2_infer'", run this cell to setup the environment:


In [ ]:
# 🚀 QUICK SETUP - Run this if you get import errors!

import os

print("🔧 Quick setup starting...")

# Check current directory
current_dir = os.getcwd()
print(f"📁 Current directory: {current_dir}")

# Check if we're in SkyReels-V2 directory
if not os.path.exists("skyreels_v2_infer"):
    print("📥 SkyReels-V2 not found, cloning repository...")
    
    # Clone the repository
    !git clone https://github.com/SkyworkAI/SkyReels-V2.git
    
    # Change to the directory
    os.chdir("/content/SkyReels-V2")
    print(f"📂 Changed to: {os.getcwd()}")
else:
    print("✅ SkyReels-V2 already exists")

# Verify the module exists
if os.path.exists("skyreels_v2_infer"):
    print("✅ skyreels_v2_infer module found!")
    
    # List contents to verify
    contents = os.listdir("skyreels_v2_infer")
    print(f"📋 Module contents: {contents}")
else:
    print("❌ skyreels_v2_infer module not found!")
    print("🔄 Try running the installation cells first")

# Install missing dependencies if needed
print("\n📦 Installing missing dependencies...")
!pip install -q decord av dashscope

print("✅ Quick setup completed!")
print("🔄 Now try importing the modules again")


In [ ]:
## ⚡ One-Click Complete Setup

**For immediate use: Run this cell to install everything and start the interface!**


In [ ]:
# ⚡ ONE-CLICK SETUP - Run this cell for complete installation!

import os
import subprocess
import sys

def run_command(cmd, description):
    """Run a command and show progress"""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ {description} completed")
        else:
            print(f"⚠️ {description} had warnings: {result.stderr[:100]}")
    except Exception as e:
        print(f"❌ {description} failed: {e}")

print("🚀 Starting complete SkyReels V2 setup...")

# 1. System dependencies
run_command("apt-get update -qq", "Updating system packages")
run_command("apt-get install -y -qq ffmpeg libsm6 libxext6 libxrender-dev libglib2.0-0", "Installing system dependencies")

# 2. Clone repository if not exists
if not os.path.exists("/content/SkyReels-V2"):
    run_command("cd /content && git clone https://github.com/SkyworkAI/SkyReels-V2.git", "Cloning SkyReels-V2 repository")

# 3. Change to directory
os.chdir("/content/SkyReels-V2")
print(f"📂 Changed to: {os.getcwd()}")

# 4. Install Python dependencies
packages = [
    "torch==2.5.1 torchvision==0.20.1",
    "diffusers>=0.31.0 transformers==4.49.0 tokenizers==0.21.1", 
    "accelerate==1.6.0 tqdm imageio easydict ftfy",
    "opencv-python==4.10.0.84 imageio-ffmpeg",
    "gradio spaces huggingface_hub",
    "numpy>=1.23.5,<2",
    "decord av dashscope",  # Video processing
    "xfuser"  # Multi-GPU support
]

for package in packages:
    run_command(f"pip install -q {package}", f"Installing {package.split()[0]}")

# 5. Verify installation
print("\n🔍 Verifying installation...")

# Check GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU: {gpu_name} ({gpu_memory:.1f}GB)")
else:
    print("❌ No GPU detected - please enable GPU runtime")

# Check modules
try:
    from skyreels_v2_infer.modules import download_model
    print("✅ SkyReels modules available")
except ImportError:
    print("❌ SkyReels modules not found")

# Check disk space
import shutil
total, used, free = shutil.disk_usage('/')
print(f"💽 Disk space: {free//1024**3:.1f}GB free / {total//1024**3:.1f}GB total")

print("\n🎉 Setup completed! You can now run the interface cells below.")
print("💡 Tip: Use 1.3B models if you have limited GPU memory or disk space.")


In [ ]:
## 🔧 Module Import Fix

If setup completed but modules weren't found, run this cell to fix the Python path:


In [ ]:
# 🔧 MODULE IMPORT FIX - Run this if modules not found after setup

import os
import sys

print("🔧 Fixing module imports...")

# 1. Ensure we're in the right directory
target_dir = "/content/SkyReels-V2"
if os.getcwd() != target_dir:
    os.chdir(target_dir)
    print(f"📂 Changed to: {os.getcwd()}")

# 2. Add to Python path
if target_dir not in sys.path:
    sys.path.insert(0, target_dir)
    print(f"📍 Added {target_dir} to Python path")

# 3. Verify directory structure
if os.path.exists("skyreels_v2_infer"):
    print("✅ skyreels_v2_infer directory found")
    
    # List contents
    contents = os.listdir("skyreels_v2_infer")
    print(f"📋 Contents: {contents[:5]}...")  # Show first 5 items
    
    # Check specific modules
    modules_to_check = ["modules", "pipelines", "__init__.py"]
    for module in modules_to_check:
        path = os.path.join("skyreels_v2_infer", module)
        if os.path.exists(path):
            print(f"✅ {module} found")
        else:
            print(f"❌ {module} missing")
else:
    print("❌ skyreels_v2_infer directory not found!")
    print("🔄 Repository may not be fully cloned")

# 4. Test import
try:
    from skyreels_v2_infer.modules import download_model
    print("✅ download_model imported successfully!")
    
    from skyreels_v2_infer.pipelines import Text2VideoPipeline, Image2VideoPipeline
    print("✅ Pipeline modules imported successfully!")
    
    print("🎉 All modules working correctly!")
    
except ImportError as e:
    print(f"❌ Import still failing: {e}")
    print("🔄 Possible solutions:")
    print("   1. Restart runtime: Runtime → Restart runtime")
    print("   2. Re-run the One-Click Setup")
    print("   3. Manual clone: !git clone https://github.com/SkyworkAI/SkyReels-V2.git")

# 5. Check disk space again
import shutil
total, used, free = shutil.disk_usage('/')
print(f"\n💽 Current disk space: {free//1024**3:.1f}GB free")


In [ ]:
## 🆘 Emergency Alternative - Direct Interface

If imports still fail, use this simplified version that downloads and runs everything directly:


In [ ]:
# 🆘 EMERGENCY DIRECT INTERFACE - Use if normal imports fail

# Install dependencies directly
!pip install -q gradio huggingface_hub

# Download and run the standalone interface
!wget -q https://raw.githubusercontent.com/SkyworkAI/SkyReels-V2/main/skyreels_gradio_interface.py -O /content/skyreels_interface.py

# Modify the script to add proper imports
import os
os.chdir("/content/SkyReels-V2")

# Direct execution
print("🚀 Starting emergency interface...")
print("📁 Current directory:", os.getcwd())

# Check if modules exist
if os.path.exists("skyreels_v2_infer"):
    print("✅ SkyReels modules found, attempting direct execution...")
    
    # Try to run the interface directly
    try:
        exec(open('/content/skyreels_interface.py').read())
    except Exception as e:
        print(f"❌ Direct execution failed: {e}")
        print("🔄 Please try restarting runtime and running One-Click Setup again")
else:
    print("❌ SkyReels-V2 not properly installed")
    print("🔄 Solutions:")
    print("   1. Restart runtime completely")
    print("   2. Re-run One-Click Setup")
    print("   3. Manual installation:")
    print("      !rm -rf /content/SkyReels-V2")
    print("      !git clone https://github.com/SkyworkAI/SkyReels-V2.git /content/SkyReels-V2")


In [ ]:
## 🔑 Hugging Face Access Token Setup

SkyReels V2 models require a Hugging Face account and access token. Set it up here:


In [ ]:
# 🔑 HUGGING FACE TOKEN SETUP

import os
from huggingface_hub import login
import getpass

print("🔑 Hugging Face Access Token Setup")
print("=" * 50)

# Check if token is already set
try:
    from huggingface_hub import HfFolder
    existing_token = HfFolder.get_token()
    if existing_token:
        print("✅ Token already configured!")
        print(f"🔐 Token: {existing_token[:8]}...{existing_token[-8:]}")
        print("🔄 To change token, continue below")
    else:
        print("❌ No token found")
except:
    print("❌ No token found")

print("\n📝 How to get your Hugging Face token:")
print("1. Go to: https://huggingface.co/settings/tokens")
print("2. Click 'New token'")
print("3. Choose 'Read' access (sufficient for model downloads)")
print("4. Copy the token")

print("\n" + "="*50)
print("🔐 Enter your Hugging Face token below:")

# Get token from user
try:
    # Try to get token securely
    token = getpass.getpass("Enter your HF token (text will be hidden): ")
    
    if not token.strip():
        print("❌ No token entered!")
    else:
        # Login to Hugging Face
        try:
            login(token=token.strip(), add_to_git_credential=True)
            print("✅ Successfully logged in to Hugging Face!")
            
            # Set environment variable for this session
            os.environ['HF_TOKEN'] = token.strip()
            os.environ['HUGGINGFACE_HUB_TOKEN'] = token.strip()
            
            print("🔐 Token saved for this session")
            
            # Test token by checking whoami
            try:
                from huggingface_hub import whoami
                user_info = whoami()
                print(f"👤 Logged in as: {user_info['name']}")
            except:
                print("⚠️  Token set but couldn't verify user info")
                
        except Exception as e:
            print(f"❌ Login failed: {e}")
            print("🔄 Please check your token and try again")
            
except Exception as e:
    print(f"❌ Error getting token: {e}")
    print("\n🔧 Alternative: Set token manually")
    print("Copy and run this with your token:")
    print("```")
    print("from huggingface_hub import login")
    print("login(token='YOUR_TOKEN_HERE')")
    print("```")

print("\n" + "="*50)
print("💡 Your token is only stored for this Colab session")
print("🔄 You'll need to re-enter it if you restart the runtime")


In [ ]:
## ⚠️ Model Availability Fix

**Important**: Some models are not yet available. Here are the ACTUAL available models:


In [ ]:
# ⚠️ MODEL AVAILABILITY CHECK

print("📋 SkyReels V2 - Currently Available Models")
print("=" * 50)

# Available models as of current date
available_models = {
    "Text-to-Video": {
        "14B-540P": "✅ Skywork/SkyReels-V2-T2V-14B-540P",
        "14B-720P": "✅ Skywork/SkyReels-V2-T2V-14B-720P",
        "1.3B-540P": "❌ Coming Soon",
        "1.3B-720P": "❌ Not Available"
    },
    "Image-to-Video": {
        "1.3B-540P": "✅ Skywork/SkyReels-V2-I2V-1.3B-540P",
        "14B-540P": "✅ Skywork/SkyReels-V2-I2V-14B-540P", 
        "14B-720P": "✅ Skywork/SkyReels-V2-I2V-14B-720P",
        "1.3B-720P": "❌ Not Available"
    },
    "Diffusion Forcing": {
        "1.3B-540P": "✅ Skywork/SkyReels-V2-DF-1.3B-540P",
        "14B-540P": "✅ Skywork/SkyReels-V2-DF-14B-540P",
        "14B-720P": "✅ Skywork/SkyReels-V2-DF-14B-720P"
    }
}

for model_type, models in available_models.items():
    print(f"\n🎬 {model_type}:")
    for size, status in models.items():
        print(f"   {size}: {status}")

print("\n" + "=" * 50)
print("💡 RECOMMENDATIONS:")
print("📱 For limited GPU memory (T4): Use Image-to-Video 1.3B-540P")
print("🚀 For Text-to-Video: Use 14B models (requires more GPU memory)")
print("🎥 For long videos: Use Diffusion Forcing models")

print("\n⚠️  CURRENT ISSUE:")
print("❌ Text-to-Video 1.3B models are not yet available!")
print("✅ Use Image-to-Video 1.3B-540P or Text-to-Video 14B instead")

print("\n🔧 Quick Fix for your error:")
print("Change the interface settings to:")
print("- Model Type: Image-to-Video") 
print("- Model Size: 1.3B")
print("- Resolution: 540P")
print("OR")
print("- Model Type: Text-to-Video")
print("- Model Size: 14B") 
print("- Resolution: 540P")


In [ ]:
## 🎨 Updated Gradio Interface (With Correct Models)

This version only shows available model combinations:


In [ ]:
# 🎨 UPDATED INTERFACE WITH CORRECT MODEL VALIDATION

import gradio as gr

# Available model combinations
AVAILABLE_MODELS = {
    ("Text-to-Video", "14B", "540P"): "Skywork/SkyReels-V2-T2V-14B-540P",
    ("Text-to-Video", "14B", "720P"): "Skywork/SkyReels-V2-T2V-14B-720P", 
    ("Image-to-Video", "1.3B", "540P"): "Skywork/SkyReels-V2-I2V-1.3B-540P",
    ("Image-to-Video", "14B", "540P"): "Skywork/SkyReels-V2-I2V-14B-540P",
    ("Image-to-Video", "14B", "720P"): "Skywork/SkyReels-V2-I2V-14B-720P",
}

class SmartSkyReelsInterface:
    def __init__(self):
        self.pipeline = None
        self.current_model = None
    
    def validate_and_generate(self, prompt, image, model_type, model_size, resolution,
                            num_frames, guidance_scale, shift, steps, fps, seed, 
                            use_offload, progress=gr.Progress()):
        
        # Validate model combination
        combo = (model_type, model_size, resolution)
        if combo not in AVAILABLE_MODELS:
            available_list = "\\n".join([f"  - {mt} {ms} {res}" 
                                       for mt, ms, res in AVAILABLE_MODELS.keys()])
            return None, f"❌ Model combination not available!\\n\\nAvailable combinations:\\n{available_list}"
        
        model_id = AVAILABLE_MODELS[combo]
        
        try:
            progress(0, "Initializing...")
            
            # Import required modules (with error handling)
            try:
                from skyreels_v2_infer.modules import download_model
                from skyreels_v2_infer.pipelines import Image2VideoPipeline, Text2VideoPipeline
                from skyreels_v2_infer.pipelines import resizecrop
            except ImportError as e:
                return None, f"❌ Module import failed: {e}\\n🔄 Please run the setup cells above first!"
            
            progress(0.1, "Loading model...")
            model_path = download_model(model_id)
            
            # Set seed
            if seed == -1:
                seed = random.randint(0, 2147483647)
            
            # Set dimensions
            height, width = (544, 960) if resolution == "540P" else (720, 1280)
            
            progress(0.3, "Initializing pipeline...")
            
            # Create pipeline if needed
            if self.current_model != model_id:
                if self.pipeline:
                    del self.pipeline
                    torch.cuda.empty_cache()
                
                if model_type == "Text-to-Video":
                    self.pipeline = Text2VideoPipeline(
                        model_path=model_path, 
                        dit_path=model_path, 
                        use_usp=False, 
                        offload=use_offload
                    )
                else:
                    self.pipeline = Image2VideoPipeline(
                        model_path=model_path, 
                        dit_path=model_path, 
                        use_usp=False, 
                        offload=use_offload
                    )
                
                self.current_model = model_id
            
            progress(0.5, "Generating video...")
            
            # Prepare inputs
            kwargs = {
                "prompt": prompt,
                "negative_prompt": "worst quality, low quality, static",
                "num_frames": num_frames,
                "num_inference_steps": steps,
                "guidance_scale": guidance_scale,
                "shift": shift,
                "generator": torch.Generator(device="cuda").manual_seed(seed),
                "height": height,
                "width": width,
            }
            
            # Add image if provided
            if image is not None:
                img = Image.fromarray(image).convert("RGB")
                if img.height > img.width:
                    height, width = width, height
                kwargs["image"] = resizecrop(img, height, width)
            
            # Generate
            with torch.cuda.amp.autocast(dtype=self.pipeline.transformer.dtype), torch.no_grad():
                frames = self.pipeline(**kwargs)[0]
            
            progress(0.9, "Saving...")
            
            # Save video
            os.makedirs("outputs", exist_ok=True)
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"video_{timestamp}_{seed}.mp4"
            path = os.path.join("outputs", filename)
            
            imageio.mimwrite(path, frames, fps=fps, quality=8)
            
            # Clear memory
            del frames
            torch.cuda.empty_cache()
            gc.collect()
            
            progress(1.0, "Done!")
            
            info = f"✅ Generated video\\nModel: {model_id}\\nSeed: {seed}\\nFile: {filename}"
            return path, info
            
        except Exception as e:
            torch.cuda.empty_cache()
            gc.collect()
            return None, f"❌ Error: {str(e)}"


# Create interface
smart_interface = SmartSkyReelsInterface()

def create_smart_interface():
    with gr.Blocks(title="SkyReels V2 - Smart Interface", theme=gr.themes.Soft()) as app:
        gr.HTML("<h1 align='center'>🎬 SkyReels V2 - Smart Interface</h1>")
        gr.HTML("<p align='center'>Only shows available model combinations</p>")
        
        with gr.Row():
            with gr.Column():
                prompt = gr.Textbox(
                    label="Prompt", 
                    lines=3,
                    value="A beautiful swan swimming gracefully in a serene lake at dawn"
                )
                
                image = gr.Image(label="Input Image (for I2V)", type="numpy")
                
                with gr.Row():
                    model_type = gr.Dropdown(
                        ["Image-to-Video", "Text-to-Video"],  # I2V first (has 1.3B option)
                        value="Image-to-Video",
                        label="Type"
                    )
                    model_size = gr.Dropdown(["1.3B", "14B"], value="1.3B", label="Size")
                
                with gr.Row():
                    resolution = gr.Dropdown(["540P", "720P"], value="540P", label="Resolution")
                    num_frames = gr.Slider(49, 121, 97, label="Frames")
                
                with gr.Row():
                    guidance_scale = gr.Slider(1, 10, 5, label="Guidance")  # Default for I2V
                    shift = gr.Slider(1, 10, 3, label="Shift")  # Default for I2V
                
                with gr.Row():
                    steps = gr.Slider(10, 100, 50, label="Steps")
                    fps = gr.Slider(8, 30, 24, label="FPS")
                
                seed = gr.Number(label="Seed (-1 = random)", value=-1)
                use_offload = gr.Checkbox(label="CPU Offload", value=True)
                
                btn = gr.Button("🎬 Generate Video", variant="primary")
                
                # Model validation info
                gr.HTML("""
                <div style='background: #f0f0f0; padding: 10px; border-radius: 5px; margin: 10px 0;'>
                <strong>Available Models:</strong><br>
                • Text-to-Video: Only 14B (540P/720P)<br>
                • Image-to-Video: 1.3B (540P) + 14B (540P/720P)<br>
                <em>💡 For limited GPU: Use Image-to-Video 1.3B 540P</em>
                </div>
                """)
            
            with gr.Column():
                video_out = gr.Video(label="Generated Video")
                info_out = gr.Textbox(label="Info", lines=4)
        
        # Auto-adjust guidance/shift based on model type
        def update_params(model_type_val):
            if model_type_val == "Text-to-Video":
                return gr.update(value=6.0), gr.update(value=8.0)
            else:
                return gr.update(value=5.0), gr.update(value=3.0)
        
        model_type.change(
            update_params,
            inputs=[model_type],
            outputs=[guidance_scale, shift]
        )
        
        btn.click(
            smart_interface.validate_and_generate,
            inputs=[prompt, image, model_type, model_size, resolution, 
                   num_frames, guidance_scale, shift, steps, fps, seed, use_offload],
            outputs=[video_out, info_out]
        )
    
    return app

# Launch interface
print("✅ Smart interface created with model validation")
smart_app = create_smart_interface()
smart_app.launch(share=True, debug=False)


In [ ]:
## 💡 Quick Solution Summary

**Das Problem:** 
❌ Der Text-to-Video 1.3B Modell ist noch nicht verfügbar (404 Error)

**Die Lösung:** 
✅ Verwende stattdessen eine verfügbare Kombination:

1. **Für begrenzte GPU Memory (T4):**
   - Modell-Typ: **Image-to-Video**
   - Modell-Größe: **1.3B**
   - Auflösung: **540P**

2. **Für Text-to-Video:**
   - Modell-Typ: **Text-to-Video**
   - Modell-Größe: **14B** (benötigt mehr GPU Memory)
   - Auflösung: **540P**

**Nächste Schritte:**
1. Führe die Modell-Verfügbarkeits-Check-Zelle aus (oben)
2. Nutze die aktualisierte Smart Interface (darunter)
3. Wähle eine verfügbare Modell-Kombination


In [ ]:
## 🔧 GPU Check and Configuration


In [ ]:
import torch
import gc

# Check GPU availability and memory
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    
    # Recommend model based on GPU memory
    if gpu_memory >= 45:
        recommended_model = "14B models (all resolutions)"
    elif gpu_memory >= 15:
        recommended_model = "1.3B models or 14B-540P with offloading"
    else:
        recommended_model = "1.3B models with heavy offloading"
    
    print(f"📝 Recommended: {recommended_model}")
else:
    print("❌ No GPU available. Please enable GPU in Runtime > Change runtime type")

# Clear cache
torch.cuda.empty_cache()
gc.collect()


In [ ]:
## 🎨 Gradio Interface Setup


In [ ]:
# Ensure we're in the right directory and dependencies are installed
import os
import sys
import gc

# Check if we're in SkyReels-V2 directory
if not os.path.exists("skyreels_v2_infer"):
    print("⚠️  skyreels_v2_infer not found!")
    print("🔄 Please run the Quick Setup cell above first!")
    print("📍 Or manually run: %cd /content/SkyReels-V2")
    raise ImportError("Please run the Quick Setup cell above to clone the repository and install dependencies")

print(f"📁 Current directory: {os.getcwd()}")

# Add current directory to Python path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import required modules
import gradio as gr
import time
import random
import imageio
import numpy as np
from PIL import Image
from diffusers.utils import load_image
import torch

# Import SkyReels modules
try:
    from skyreels_v2_infer.modules import download_model
    from skyreels_v2_infer.pipelines import Image2VideoPipeline, Text2VideoPipeline
    from skyreels_v2_infer.pipelines import PromptEnhancer, resizecrop
    print("✅ All SkyReels modules imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("🔄 Please run the Quick Setup cell above first!")
    print("📦 Or install missing dependencies: !pip install decord av dashscope")
    raise

class SkyReelsV2Interface:
    def __init__(self):
        self.current_pipeline = None
        self.current_model_id = None
        
    def generate_video(self, 
                      prompt, 
                      image,
                      model_type,
                      model_size,
                      resolution,
                      num_frames,
                      guidance_scale,
                      shift,
                      inference_steps,
                      fps,
                      seed,
                      use_prompt_enhancer,
                      use_offload,
                      progress=gr.Progress()):
        
        try:
            progress(0, desc="Initializing...")
            
            # Determine model ID
            if model_type == "Text-to-Video":
                if model_size == "14B":
                    model_id = f"Skywork/SkyReels-V2-T2V-14B-{resolution}"
                else:
                    model_id = f"Skywork/SkyReels-V2-T2V-1.3B-{resolution}"
            else:  # Image-to-Video
                if model_size == "14B":
                    model_id = f"Skywork/SkyReels-V2-I2V-14B-{resolution}"
                else:
                    model_id = f"Skywork/SkyReels-V2-I2V-1.3B-{resolution}"
            
            progress(0.1, desc="Downloading model...")
            model_path = download_model(model_id)
            
            # Set seed
            if seed == -1:
                seed = random.randint(0, 2147483647)
            
            # Set dimensions
            if resolution == "540P":
                height, width = 544, 960
            else:  # 720P
                height, width = 720, 1280
            
            progress(0.2, desc="Processing prompt...")
            prompt_input = prompt
            
            # Enhance prompt if requested and no image provided
            if use_prompt_enhancer and image is None:
                try:
                    progress(0.25, desc="Enhancing prompt...")
                    enhancer = PromptEnhancer()
                    prompt_input = enhancer(prompt)
                    del enhancer
                    torch.cuda.empty_cache()
                    print(f"Enhanced prompt: {prompt_input}")
                except Exception as e:
                    print(f"Prompt enhancement failed: {e}")
                    prompt_input = prompt
            
            progress(0.3, desc="Loading pipeline...")
            
            # Initialize pipeline
            if image is None:
                if self.current_model_id != model_id or self.current_pipeline is None:
                    if self.current_pipeline:
                        del self.current_pipeline
                        torch.cuda.empty_cache()
                    
                    self.current_pipeline = Text2VideoPipeline(
                        model_path=model_path, 
                        dit_path=model_path, 
                        use_usp=False, 
                        offload=use_offload
                    )
                    self.current_model_id = model_id
            else:
                if self.current_model_id != model_id or self.current_pipeline is None:
                    if self.current_pipeline:
                        del self.current_pipeline
                        torch.cuda.empty_cache()
                    
                    self.current_pipeline = Image2VideoPipeline(
                        model_path=model_path, 
                        dit_path=model_path, 
                        use_usp=False, 
                        offload=use_offload
                    )
                    self.current_model_id = model_id
                
                # Process input image
                input_image = Image.fromarray(image).convert("RGB")
                image_width, image_height = input_image.size
                if image_height > image_width:
                    height, width = width, height
                input_image = resizecrop(input_image, height, width)
            
            progress(0.5, desc="Generating video...")
            
            # Prepare generation parameters
            negative_prompt = "Bright tones, overexposed, static, blurred details, subtitles, style, works, paintings, images, static, overall gray, worst quality, low quality, JPEG compression residue, ugly, incomplete, extra fingers, poorly drawn hands, poorly drawn faces, deformed, disfigured, misshapen limbs, fused fingers, still picture, messy background, three legs, many people in the background, walking backwards"
            
            kwargs = {
                "prompt": prompt_input,
                "negative_prompt": negative_prompt,
                "num_frames": num_frames,
                "num_inference_steps": inference_steps,
                "guidance_scale": guidance_scale,
                "shift": shift,
                "generator": torch.Generator(device="cuda").manual_seed(seed),
                "height": height,
                "width": width,
            }
            
            if image is not None:
                kwargs["image"] = input_image
            
            # Generate video
            with torch.cuda.amp.autocast(dtype=self.current_pipeline.transformer.dtype), torch.no_grad():
                video_frames = self.current_pipeline(**kwargs)[0]
            
            progress(0.9, desc="Saving video...")
            
            # Save video
            os.makedirs("outputs", exist_ok=True)
            current_time = time.strftime("%Y%m%d_%H%M%S", time.localtime())
            video_filename = f"skyreels_v2_{current_time}_{seed}.mp4"
            video_path = os.path.join("outputs", video_filename)
            
            imageio.mimwrite(video_path, video_frames, fps=fps, quality=8, output_params=["-loglevel", "error"])
            
            progress(1.0, desc="Complete!")
            
            return video_path, f"✅ Video generated successfully!\\nSeed: {seed}\\nFrames: {len(video_frames)}\\nResolution: {width}x{height}"
            
        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"
            print(error_msg)
            return None, error_msg

# Create interface instance
skyreels_interface = SkyReelsV2Interface()

print("✅ Interface setup completed!")


In [ ]:
## 🌟 Launch Interactive Interface


In [ ]:
# Create Gradio interface
def create_interface():
    with gr.Blocks(title="SkyReels V2 - Video Generator", theme=gr.themes.Soft()) as demo:
        gr.HTML("""
        <div style="text-align: center; margin-bottom: 20px;">
            <h1>🎬 SkyReels V2 - Infinite-Length Film Generator</h1>
            <p>Generate high-quality videos with state-of-the-art AI models</p>
        </div>
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📝 Input Configuration")
                
                prompt = gr.Textbox(
                    label="Prompt",
                    placeholder="Describe the video you want to generate...",
                    lines=3,
                    value="A graceful white swan with a curved neck and delicate feathers swimming in a serene lake at dawn, its reflection perfectly mirrored in the still water as mist rises from the surface."
                )
                
                image = gr.Image(
                    label="Input Image (Optional - for Image-to-Video)",
                    type="numpy"
                )
                
                with gr.Row():
                    model_type = gr.Dropdown(
                        choices=["Text-to-Video", "Image-to-Video"],
                        value="Text-to-Video",
                        label="Generation Type"
                    )
                    
                    model_size = gr.Dropdown(
                        choices=["1.3B", "14B"],
                        value="1.3B",
                        label="Model Size"
                    )
                
                with gr.Row():
                    resolution = gr.Dropdown(
                        choices=["540P", "720P"],
                        value="540P",
                        label="Resolution"
                    )
                    
                    num_frames = gr.Slider(
                        minimum=49,
                        maximum=121,
                        value=97,
                        step=1,
                        label="Number of Frames"
                    )
                
                gr.Markdown("### ⚙️ Advanced Settings")
                
                with gr.Row():
                    guidance_scale = gr.Slider(
                        minimum=1.0,
                        maximum=10.0,
                        value=6.0,
                        step=0.5,
                        label="Guidance Scale"
                    )
                    
                    shift = gr.Slider(
                        minimum=1.0,
                        maximum=10.0,
                        value=8.0,
                        step=0.5,
                        label="Shift"
                    )
                
                with gr.Row():
                    inference_steps = gr.Slider(
                        minimum=10,
                        maximum=100,
                        value=50,
                        step=1,
                        label="Inference Steps"
                    )
                    
                    fps = gr.Slider(
                        minimum=8,
                        maximum=30,
                        value=24,
                        step=1,
                        label="FPS"
                    )
                
                seed = gr.Number(
                    label="Seed (-1 for random)",
                    value=-1,
                    precision=0
                )
                
                with gr.Row():
                    use_prompt_enhancer = gr.Checkbox(
                        label="Enhance Prompt",
                        value=False
                    )
                    
                    use_offload = gr.Checkbox(
                        label="CPU Offload (reduces VRAM)",
                        value=True
                    )
                
                generate_btn = gr.Button(
                    "🎬 Generate Video",
                    variant="primary",
                    size="lg"
                )
            
            with gr.Column(scale=1):
                gr.Markdown("### 🎥 Generated Video")
                
                output_video = gr.Video(
                    label="Generated Video",
                    height=400
                )
                
                output_info = gr.Textbox(
                    label="Generation Info",
                    lines=4,
                    interactive=False
                )
                
                gr.Markdown("""
                ### 💡 Tips:
                - **1.3B models**: Faster, lower VRAM usage (~15GB)
                - **14B models**: Higher quality, more VRAM (~45GB)
                - **540P**: 544x960 resolution, 97 frames max
                - **720P**: 720x1280 resolution, 121 frames max
                - **CPU Offload**: Reduces VRAM usage but slower
                - **Image-to-Video**: Upload an image for I2V generation
                """)
        
        # Auto-adjust parameters based on model type
        def update_params(model_type):
            if model_type == "Image-to-Video":
                return gr.update(value=5.0), gr.update(value=3.0)  # guidance_scale, shift
            else:
                return gr.update(value=6.0), gr.update(value=8.0)
        
        model_type.change(
            update_params,
            inputs=[model_type],
            outputs=[guidance_scale, shift]
        )
        
        # Auto-adjust frames based on resolution
        def update_frames(resolution):
            if resolution == "720P":
                return gr.update(maximum=121, value=121)
            else:
                return gr.update(maximum=97, value=97)
        
        resolution.change(
            update_frames,
            inputs=[resolution],
            outputs=[num_frames]
        )
        
        # Generate video
        generate_btn.click(
            skyreels_interface.generate_video,
            inputs=[
                prompt, image, model_type, model_size, resolution,
                num_frames, guidance_scale, shift, inference_steps,
                fps, seed, use_prompt_enhancer, use_offload
            ],
            outputs=[output_video, output_info]
        )
    
    return demo

# Launch the interface
demo = create_interface()
demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860,
    show_error=True
)


In [ ]:
## 📚 Example Usage

Here are some example prompts and settings to get you started:


In [ ]:
# Example prompts for different scenarios
example_prompts = {
    "Nature": "A majestic waterfall cascading down moss-covered rocks in a lush forest, with sunbeams filtering through the canopy and mist rising from the pool below.",
    
    "Urban": "A bustling city street at night with neon lights reflecting on wet pavement, people walking with umbrellas, and cars passing by with light trails.",
    
    "Portrait": "A close-up portrait of a person with curly hair sitting by a window, soft natural light illuminating their face, gentle smile, looking thoughtfully outside.",
    
    "Abstract": "Colorful paint drops falling into water in slow motion, creating beautiful ripples and color mixing patterns, artistic and mesmerizing.",
    
    "Animals": "A family of dolphins jumping gracefully out of crystal clear ocean water at sunset, with golden light reflecting on the waves."
}

print("📚 Example Prompts:")
for category, prompt in example_prompts.items():
    print(f"\n{category}:")
    print(f"  {prompt}")

print("\n💡 Pro Tips:")
print("- Use descriptive language with visual details")
print("- Specify lighting, camera angles, and movement")
print("- Mention colors, textures, and atmosphere")
print("- Keep prompts between 20-100 words for best results")


In [ ]:
## 🔧 Troubleshooting

Common issues and solutions:


In [ ]:
def check_system_status():
    """Check system status and provide troubleshooting info"""
    
    print("=== System Status Check ===")
    
    # GPU Check
    if torch.cuda.is_available():
        gpu_memory_allocated = torch.cuda.memory_allocated(0) / 1024**3
        gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        gpu_memory_free = gpu_memory_total - gpu_memory_allocated
        
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {gpu_memory_allocated:.2f}GB / {gpu_memory_total:.2f}GB used")
        print(f"🆓 Free Memory: {gpu_memory_free:.2f}GB")
        
        if gpu_memory_free < 10:
            print("⚠️  Warning: Low GPU memory. Consider:")
            print("   - Using 1.3B model instead of 14B")
            print("   - Enabling CPU offload")
            print("   - Reducing number of frames")
            print("   - Using 540P instead of 720P")
    else:
        print("❌ No GPU detected. Please enable GPU runtime.")
    
    # Memory cleanup
    print("\n🧹 Cleaning up memory...")
    torch.cuda.empty_cache()
    gc.collect()
    print("✅ Memory cleanup completed")
    
    # Disk space
    import shutil
    disk_usage = shutil.disk_usage('.')
    free_gb = disk_usage.free / 1024**3
    print(f"💽 Free Disk Space: {free_gb:.2f}GB")
    
    if free_gb < 5:
        print("⚠️  Warning: Low disk space. Clean up old files if needed.")

# Run system check
check_system_status()

print("\n🆘 Common Issues & Solutions:")
print("\n1. Out of Memory Error:")
print("   - Use smaller model (1.3B instead of 14B)")
print("   - Enable CPU offload")
print("   - Reduce number of frames")
print("   - Use 540P resolution")

print("\n2. Generation Takes Too Long:")
print("   - Reduce inference steps (20-30)")
print("   - Use smaller model")
print("   - Disable prompt enhancer")

print("\n3. Poor Video Quality:")
print("   - Use larger model (14B)")
print("   - Increase inference steps (50-100)")
print("   - Improve prompt description")
print("   - Adjust guidance scale (5-8)")

print("\n4. Installation Issues:")
print("   - Restart runtime and run cells again")
print("   - Check GPU is enabled in runtime settings")
print("   - Clear output and restart if needed")
